### Disclaimer
- notebook first made by naver-boostcamp
- First edited by cuffyluv - 251208

https://stages.ai/competitions/378/board/community/post/2970

In [1]:
import json
import numpy as np
from collections import Counter
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

In [2]:
# load json annotation = {dataset file 경로}

annotation = "../../dataset/train.json"
with open(annotation) as f: 
    data = json.load(f)

var = [(ann['image_id'], ann['category_id']) for ann in data['annotations']]
X = np.ones((len(data['annotations']),1))
y = np.array([v[1] for v in var])
groups = np.array([v[0] for v in var])

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=411)

for train_idx, val_idx in cv.split(X, y, groups):
    print("TRAIN:", groups[train_idx])
    print(" ", y[train_idx])
    print(" TEST:", groups[val_idx])
    print(" ", y[val_idx])

TRAIN: [   0    1    1 ... 4881 4881 4881]
  [0 3 7 ... 7 1 7]
 TEST: [   6   13   13 ... 4882 4882 4882]
  [1 6 7 ... 0 1 1]
TRAIN: [   0    1    1 ... 4882 4882 4882]
  [0 3 7 ... 0 1 1]
 TEST: [   5    5    5 ... 4876 4876 4878]
  [7 0 0 ... 0 2 0]
TRAIN: [   0    3    3 ... 4882 4882 4882]
  [0 2 6 ... 0 1 1]
 TEST: [   1    1    1 ... 4877 4877 4880]
  [3 7 4 ... 7 7 0]
TRAIN: [   1    1    1 ... 4882 4882 4882]
  [3 7 4 ... 0 1 1]
 TEST: [   0    3    3 ... 4881 4881 4881]
  [0 2 6 ... 7 1 7]
TRAIN: [   0    1    1 ... 4882 4882 4882]
  [0 3 7 ... 0 1 1]
 TEST: [   4    4    4 ... 4868 4872 4872]
  [1 1 1 ... 2 4 6]


In [5]:

# check distribution

def get_distribution(y):
    y_distr = Counter(y)
    y_vals_sum = sum(y_distr.values())
    return [f'{y_distr[i]/y_vals_sum:.2%}' for i in range(np.max(y) + 1)]

distrs = [get_distribution(y)]
index = ['training set']

for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X,y, groups)):
    train_y, val_y = y[train_idx], y[val_idx]
    train_gr, val_gr = groups[train_idx], groups[val_idx]

    assert len(set(train_gr) & set(val_gr)) == 0 
    distrs.append(get_distribution(train_y))

    distrs.append(get_distribution(val_y))
    index.append(f'train - fold{fold_ind}')
    index.append(f'val - fold{fold_ind}')

categories = [d['name'] for d in data['categories']]
pd.DataFrame(distrs, index=index, columns = [categories[i] for i in range(np.max(y) + 1)])

,General trash,Paper,Paper pack,Metal,Glass,Plastic,Styrofoam,Plastic bag,Battery,Clothing
training set,17.14%,27.45%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
train - fold0,16.96%,27.45%,3.79%,4.13%,4.48%,12.61%,5.51%,22.28%,0.77%,2.02%
val - fold0,17.85%,27.42%,4.23%,3.70%,3.26%,13.15%,5.25%,22.77%,0.35%,2.02%
train - fold1,17.14%,27.24%,4.01%,3.98%,4.28%,12.77%,5.38%,22.32%,0.67%,2.20%
val - fold1,17.12%,28.17%,3.41%,4.26%,4.12%,12.51%,5.72%,22.57%,0.73%,1.38%
train - fold2,17.31%,27.39%,3.83%,4.08%,4.13%,12.80%,5.14%,22.68%,0.69%,1.94%
val - fold2,16.42%,27.68%,4.05%,3.88%,4.70%,12.36%,6.76%,21.12%,0.69%,2.35%
train - fold3,17.30%,27.47%,3.87%,4.06%,4.22%,12.63%,5.49%,22.39%,0.63%,1.95%
val - fold3,16.50%,27.36%,3.88%,3.99%,4.33%,13.07%,5.33%,22.30%,0.92%,2.32%
train - fold4,16.97%,27.67%,3.88%,3.97%,4.10%,12.77%,5.76%,22.20%,0.68%,2.00%


## 위의 베이스를 보고서 새로 validation을 저장하는 코드

In [ ]:
import json
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold

# 원본 annotation 파일 (전체 train.json)
annotation_path = "../../dataset/train.json"

with open(annotation_path, "r") as f:
    data = json.load(f)

# annotations에서 label(y)과 group(image_id) 추출
annotations = data['annotations']
y = np.array([ann['category_id'] for ann in annotations])
groups = np.array([ann['image_id'] for ann in annotations])

X = np.ones((len(annotations), 1))   # 의미 없는 placeholder
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=411)


In [ ]:
import os

save_dir = "../../dataset/folds/"
os.makedirs(save_dir, exist_ok=True)

for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    print(f"\n===== Fold {fold_idx} =====")

    # 이번 fold에서 사용하는 image_id 목록 추출
    train_img_ids = set(groups[train_idx])
    val_img_ids = set(groups[val_idx])

    # 이미지 중복 없음 확인
    assert len(train_img_ids & val_img_ids) == 0

    # 이미지 정보 가져오기
    images = data['images']
    annotations = data['annotations']

    # fold별 이미지/annotation 분할
    train_images = [img for img in images if img['id'] in train_img_ids]
    val_images   = [img for img in images if img['id'] in val_img_ids]

    train_annotations = [ann for ann in annotations if ann['image_id'] in train_img_ids]
    val_annotations   = [ann for ann in annotations if ann['image_id'] in val_img_ids]

    # json 형태로 저장
    train_json = {
        "images": train_images,
        "annotations": train_annotations,
        "categories": data['categories']
    }

    val_json = {
        "images": val_images,
        "annotations": val_annotations,
        "categories": data['categories']
    }

    # 파일 저장
    with open(f"{save_dir}/train_fold{fold_idx}.json", "w") as f:
        json.dump(train_json, f, indent=2)

    with open(f"{save_dir}/val_fold{fold_idx}.json", "w") as f:
        json.dump(val_json, f, indent=2)

    print(f"Saved train_fold{fold_idx}.json  ({len(train_images)} images)")
    print(f"Saved val_fold{fold_idx}.json    ({len(val_images)} images)")



===== Fold 0 =====
Saved train_fold0.json  (3914 images)
Saved val_fold0.json    (969 images)

===== Fold 1 =====
Saved train_fold1.json  (3906 images)
Saved val_fold1.json    (977 images)

===== Fold 2 =====
Saved train_fold2.json  (3901 images)
Saved val_fold2.json    (982 images)

===== Fold 3 =====
Saved train_fold3.json  (3902 images)
Saved val_fold3.json    (981 images)

===== Fold 4 =====
Saved train_fold4.json  (3909 images)
Saved val_fold4.json    (974 images)
